# Time widget

* ~Not sure what widget options would be best and how to write a performant callback that selects and processes values for each timestep~ More widget option would still be good tho
* A dropdown selecting the properties to use in the time widget would be nice too
* Make a toggle for bottom grid
* Update grid `pts = grid.points.copy() plotter.update_coordinates(raveled_points)`

In [8]:
import pyvista as pv
import numpy as np
import xarray as xr
from JulesD3D.utils import quickDF, easyTimeFormat
from JulesD3D.processNetCDF import fixMeshGrid, addVectorSum, addDepth, makeVelocity, addUnderlayerCoords
from JulesD3D.plotPyVista import makeBottomSurface, makeStructuredGridDepth
from cmocean.cm import deep
import matplotlib.pyplot as plt
from JulesD3D.utils import ncFilePicker, folderPicker
from os import path

In [13]:
# parent_folder = '/Users/julesblom/ThesisResults/'
parent_folder = '/Volumes/Seagate Expansion Drive/FinalResults/'

scenario_folder = folderPicker(parent_folder)
scenario_folder

Select(description='Scenario folder:', layout=Layout(height='120px', width='100%'), options=('/Volumes/Seagate…

In [14]:
nc_filename = ncFilePicker(scenario_folder.value)

In [15]:
nc_filename

Select(description='NetCDF file:', layout=Layout(height='120px', width='100%'), options=('/Volumes/Seagate Exp…

In [19]:
basename = path.basename(nc_filename.value)
plot_title, _ = path.splitext(basename)
plot_title

'Slope1.25_Run50_detailed'

In [20]:
trim = xr.open_dataset(nc_filename.value) # Open with xarray : DataSet : A collection of DataArrays

OSError: [Errno -101] NetCDF: HDF error: b'/Volumes/Seagate Expansion Drive/FinalResults/Slope1.25/Slope1.25_Run50_detailed.nc'

In [6]:
if 'velocity' not in trim:
    trim = fixMeshGrid(trim, mystery_flag=True)
    trim = addDepth(trim)
    trim = makeVelocity(trim)
    trim = addUnderlayerCoords(trim)
else:
    print("DataSet is preprocessed, 'velocity' is already in DataSet")

------ Fixing mesh grid, assuming a uniform grid ------
x_gridstep:	 200.0 m
y_gridstep:	 200.0 m
width:		 26000.0 m
length:		 36200.0 m
132 x 182 grid


### Process data vars

In [7]:
concentration = trim.R1.transpose('time', 'LSTSCI', 'M', 'N', 'KMAXOUT_RESTR')
# concentration_sand = concentration.isel(LSTSCI=0)
concentration_silt = concentration.isel(LSTSCI=1)
# concentration_silt_at_time = concentration_silt.isel(time=5)
# concentration_silt_at_time = concentration_silt_at_time.assign_attrs({'long_name': 'Silt concentrations percentage per layer in zeta point',
#                                                               'units': '%',
#                                                               'grid': 'grid',
#                                                               'location': 'face'})

In [8]:
# Reorder velocity dims to match x, y, z
velocity = trim.velocity.transpose('time', 'M', 'N', 'KMAXOUT_RESTR')
velocity.dims

('time', 'M', 'N', 'KMAXOUT_RESTR')

In [9]:
density = trim.RHO.transpose('time', 'M', 'N', 'KMAXOUT_RESTR')
density.dims

('time', 'M', 'N', 'KMAXOUT_RESTR')

### Turn Delft3D output into PyVista grid

In [10]:
bottom_surface = makeBottomSurface(trim, timestep=0)
bottom_surface

StructuredGrid (0x11e20fbb0)
  N Cells:	23091
  N Points:	23400
  X Bounds:	2.000e+02, 2.600e+04
  Y Bounds:	3.000e+02, 3.610e+04
  Z Bounds:	-6.999e+02, -0.000e+00
  Dimensions:	130, 180, 1
  N Arrays:	1

In [11]:
depth_centers_grid = makeStructuredGridDepth(trim, keyword='SIG_LYR')

'depth' DataArray already found in DataSet!
xyz_layers.shape (1921920, 3)


In [12]:
plotting_options = dict(lighting=True, ambient=0.9, specular=0.1, specular_power=1) #show_edges=False ,

depth_sargs = dict(
    title_font_size=30,
    label_font_size=28,
    shadow=True,
    n_labels=5,
    fmt="%.1f",
    font_family="arial",
    vertical=True,
#     interactive=True
)

sargs = dict(
    title_font_size=30,
    label_font_size=28,
    shadow=True,
    n_labels=6,
    fmt="%.1f",
    font_family="arial",
    interactive=True
)

outline = depth_centers_grid.outline()

# Time slider Plots (wow!)

In [13]:
initial_timestep = 0
time_outputstep = trim.time[initial_timestep].values
time_lims = [0, trim.time.size]

## Density

In [14]:
depth_centers_grid['Density'] = density.isel(time=initial_timestep).values.ravel()
density_clim = (1020, 1040)

### Sideview

In [35]:
# make a generic callback function and pass in keyword, but how?
def sliceCallback(val):
    outputstep = int(val)
    Density = density.isel(time=outputstep).values.ravel()

    print(Density.max())

    p.update_scalars(Density, mesh=depth_centers_grid) 

In [29]:
channel_index = int(trim.M.size/2)
channel_x = trim.XZ.isel(N=0, M=channel_index).values # trim.XZ.max()/2

In [36]:
# channel_slice = depth_centers_grid.slice_orthogonal(x=channel_x)
# channel_slice.get_data_range('Density')

In [37]:
p = pv.Plotter(notebook=False)
p.add_mesh_slice(depth_centers_grid, normal='x', scalars='Density', cmap="viridis", clim=(1025, 1035), scalar_bar_args=sargs)
p.add_slider_widget(sliceCallback, time_lims, title="Outputstep", value=initial_timestep, event_type='end')
p.enable_depth_peeling(10)
p.show_grid()
p.set_scale(zscale=25)
p.show(cpos='yz')

{'val': 0.0}
1025.0
{'val': 3.4626166780473784}
1035.9907
{'val': 5.059008842608524}
1027.3424
{'val': 2.134644434961117}
1050.5481
{'val': 3.023832390034617}
1035.9907
{'val': 5.923042353213768}
1027.3424
{'val': 7.019261045759118}
1025.7471


[(27591.66788713736, 17449.350867220826, -464.02098869941386),
 (18452.986103036164, 17907.193056565302, -502.6124904789957),
 (-0.10714002114013327, -0.04563915826319837, 0.9931958936197424)]

### Interactive 3D Density

In [35]:
opacity_trick = [0] * 6 + [0.9] * 15 # should 'map' to steps of clim 

In [34]:
# make a generic callback function and pass
def densityCallback(val):
    p.remove_actor(initTimeActor)
    outputstep = int(val)
    Density = density.isel(time=outputstep).values.ravel()

    print(Density.max())

    p.update_scalars(Density, mesh=depth_centers_grid) 
    
    time_outputstep = trim.time.isel(time=outputstep).values
    timeActor = p.add_text(f"Density at {easyTimeFormat(time_outputstep)}", position='upper_left')

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(outline)
p.add_mesh(bottom_surface, cmap=deep, opacity=0.3, scalar_bar_args=depth_sargs)
p.add_mesh(depth_centers_grid, scalars='Density', clim=density_clim, opacity=opacity_trick, scalar_bar_args=sargs, **plotting_options)
initTimeActor = p.add_text(f"Density at {easyTimeFormat(time_outputstep)}", position='upper_left')
p.add_slider_widget(densityCallback, time_lims, title="Outputstep", value=initial_timestep, event_type='end')
p.enable_depth_peeling()
p.show_grid()
p.set_scale(zscale=25)
panel = p.show()

### Interactive density plane
TODO

In [ ]:
# slices = mesh.slice_orthogonal(x=20, y=20, z=30)
slices.plot(cmap=cmap)


# Interactive silt concentrations

In [ ]:
depth_centers_grid['Silt_concentration'] = concentration_silt.isel(time=initial_timestep).values.ravel()

In [ ]:
concentration_silt.isel(time=-10).max()

In [ ]:
concentration_silt.max()

In [ ]:
def siltCallback(val):
    p.remove_actor(initTimeActor)
    outputstep = int(val)
    Silt_concentration = concentration_silt.isel(time=outputstep).values.ravel()
    
    print(Silt_concentration.max())

    p.update_scalars(Silt_concentration, mesh=depth_centers_grid) 
    
    time_outputstep = trim.time.isel(time=outputstep).values
    timeActor = p.add_text(f"Silt conc at {easyTimeFormat(time_outputstep)}", position='upper_left')

In [ ]:
silt_clim = (0.1, 5)
silt_opacity_trick = [0] * 6 + [0.9] * 15 # should 'map' to steps of clim 

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(outline)
p.add_mesh(bottom_surface, cmap=deep, opacity=0.3)
p.add_mesh(depth_centers_grid, scalars='Silt_concentration', clim=silt_clim, opacity=silt_opacity_trick, scalar_bar_args=sargs, **plotting_options)
initTimeActor = p.add_text(f"Silt concentration at {easyTimeFormat(time_outputstep)}", position='upper_left')
p.add_slider_widget(siltCallback, time_lims, title="Outputstep", value=initial_timestep, event_type='end')
p.enable_depth_peeling()
p.show_grid()
p.set_scale(zscale=25)
panel = p.show()